![](img/cabecera.png)


# **Taller AWS: RDS MySQL - Extra Opcional**

### Paquetes requeridos  

Librería de Python necesaria para establecer conexión a una base de datos MySQL  

In [1]:
!pip install "pymysql==1.0.2"


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 1.8 MB/s eta 0:00:00-:--:--

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 1.3 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


### Credenciales  

Habitualmente para acceder a cualquier BD necesitaremos al menos tres datos:  

1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [4]:
host = "borrar-database-3.c9mwoowk2nth.eu-north-1.rds.amazonaws.com"
username = "admin"
password = "12345678martes"
port = 3306

### Conexion BD

In [5]:
import pymysql

In [3]:
'''
pymysql.cursors.Dictcursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
                     )
# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [4]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta-
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries que modifiquen la base de datos de alguna manera:
de insert de datos, creación de tablas, etc 
'''

cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print (f"MysQL version: {version}")

MysQL version: {'VERSION()': '8.0.41'}


### Creación de DB

In [5]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [6]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Seleccionar la BD

In [8]:
# Para usar la BD recien creada
cursor.connection.commit()
use_db = '''USE country_database'''
cursor.execute(use_db)

0

### Creación de tablas

In [9]:
'''
Creamos una tabla 
Esto da error porque no le hemos dicho al servidor que BD
queremos usar
'''

# Crear una tabla que se llame country
create_table = '''
    CREATE TABLE country (
        id INT NOT NULL auto_increment,
        name TEXT,
        continent TEXT,
        population INT,
        gdp DOUBLE,
        primary key (id)
    )
'''
cursor.execute(create_table)

0

In [ ]:
# Si queremos borrar alguna tabla 
# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)

### Insertar datos

In [11]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp)
VALUES ('Spain','Europe', 48, 1.62)
'''
cursor.execute(insert_data)

1

In [12]:
name = "Italy"
continent = "Europe"
population = 59
gdp = 2.3

In [13]:
insert_data = f'''
INSERT INTO country(name,continent,population,gdp) VALUES ("{name}","{continent}",{population},{gdp})
'''

In [14]:
cursor.execute(insert_data)

1

### Leer datos

In [15]:
query = '''
SELECT * FROM country WHERE name LIKE "%pa%"
'''
cursor.execute(query)

1

In [16]:
print(cursor.fetchall())

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 48, 'gdp': 1.62}]


### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [17]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [19]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

[{'id': 1, 'name': 'Spain', 'continent': 'Europe', 'population': 48, 'gdp': 1.62}, {'id': 2, 'name': 'Italy', 'continent': 'Europe', 'population': 59, 'gdp': 2.3}]


### Insertar datos de un CSV

In [ ]:
# Creamos el CSV
import pandas as pd

df= pd.read_csv('data/country_data.csv', sep=';')

In [21]:
df

,name,continent,population,gdp
0,France,Europe,68,3.05
1,Japan,Asia,124,4.20
2,Argentina,America,45,0.65
3,Brazil,America,211,2.17


In [33]:
for datos in zip(*[valor.values() for clave,valor in df.to_dict().items()]):
    print(datos)

('France', 'Europe', 68, 3.05)
('Japan', 'Asia', 124, 4.2)
('Argentina', 'America', 45, 0.65)
('Brazil', 'America', 211, 2.17)


In [34]:
for datos in zip(*[valor.values() for clave,valor in df.to_dict().items()]):
    insert = "INSERT INTO country(name,continent,population,gdp) VALUES ('%s','%s',%d,%.1f)" \
    %(datos[0],datos[1],datos[2],datos[3])
    cursor.execute(insert)

In [35]:
query = "SELECT * FROM country"
cursor.execute(query)
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 48,
  'gdp': 1.62},
 {'id': 2,
  'name': 'Italy',
  'continent': 'Europe',
  'population': 59,
  'gdp': 2.3},
 {'id': 3,
  'name': 'France',
  'continent': 'Europe',
  'population': 68,
  'gdp': 3.0},
 {'id': 4,
  'name': 'Japan',
  'continent': 'Asia',
  'population': 124,
  'gdp': 4.2},
 {'id': 5,
  'name': 'Argentina',
  'continent': 'America',
  'population': 45,
  'gdp': 0.7},
 {'id': 6,
  'name': 'Brazil',
  'continent': 'America',
  'population': 211,
  'gdp': 2.2}]

In [36]:
db.commit()

### Recuerda cerrar la conexión cuando acabemos

In [ ]:
db.close()

### Alternativa usando sqlalchemy

In [37]:
# Utilizando sqlalchemy
from sqlalchemy import create_engine
# import pymysql

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = "country_database"))

In [41]:
df.to_sql(name = "country_new", con= engine, if_exists= "append", index = False)

4

In [42]:
db.commit()

In [43]:
query = "SELECT * FROM country_new"
cursor.execute(query)
cursor.fetchall()

[{'name': 'France', 'continent': 'Europe', 'population': 68, 'gdp': 3.05},
 {'name': 'Japan', 'continent': 'Asia', 'population': 124, 'gdp': 4.2},
 {'name': 'Argentina', 'continent': 'America', 'population': 45, 'gdp': 0.65},
 {'name': 'Brazil', 'continent': 'America', 'population': 211, 'gdp': 2.17}]

### Y de nuevo, recuerda cerrar la conexión cuando acabemos

In [45]:
conn = engine.connect()
conn.close()